# SparkからTensorFlowへのデータ変換の簡素化

このノートブックでは、Databricks上での以下のワークフローをデモします。
1. Sparkによるデータロード
1. petastrom`spark_dataset_converter`による、SparkデータフレームからTensorFlowデータセットへの変換
1. シングルノードのTensorFlowモデルのトレーニングのためのデータ投入
1. 分散ハイパーパラメーターチューニング関数へのデータ投入
1. 分散TensorFlowモデルのトレーニングのためのデータ投入

このノートブックで用いる例は、[transfer learning tutorial from TensorFlow](https://www.tensorflow.org/tutorials/images/transfer_learning)をベースにしています。事前学習済みの[MobileNetV2](https://www.tensorflow.org/api_docs/python/tf/keras/applications/MobileNetV2)モデルを花のデータセットに適用します。



### 要件
1. Databricks Runtime 7.0 ML
2. ノードタイプ：1つのドライバーおよび2つのワーカーノード。GPUインスタンスを使うことをお勧めします。

<table>
  <tr><th>作者</th><th>Databricks Japan</th></tr>
  <tr><td>日付</td><td>2021/08/30</td></tr>
  <tr><td>バージョン</td><td>1.0</td></tr>
  <tr><td>クラスター</td><td>8.3ML</td></tr>
</table>
<img style="margin-top:25px;" src="https://jixjiadatabricks.blob.core.windows.net/images/databricks-logo-small-new.png" width="140">

In [0]:
from pyspark.sql.functions import col

from petastorm.spark import SparkDatasetConverter, make_spark_converter

import io
import numpy as np
import tensorflow as tf
from PIL import Image
from petastorm import TransformSpec
from tensorflow import keras
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK

import horovod.tensorflow.keras as hvd
from sparkdl import HorovodRunner

In [0]:
IMG_SHAPE = (224, 224, 3)
BATCH_SIZE = 32
NUM_EPOCHS = 5

## 1. Sparkによるデータロード

### 花のデータセット

サンプルデータセットとして、ここではTenforFlowチームによって提供される[flowers dataset](https://www.tensorflow.org/datasets/catalog/tf_flowers)を使用します。これには、それぞれがクラスを表現する5つのサブディレクトリに花の写真が格納されています。これは、簡単にアクセスできるようにDatabricksデータセットのディレクトリ`dbfs:/databricks-datasets/flower_photos`に格納されています。

この例では、バイナリーファイルデータソースを用いて前処理済みの花のデータセットを保持する花のテーブルをロードします。ここでは約900のトレーニング画像、約100の検証画像から構成される花のデータセットの小規模なサブセットを使います。

In [0]:
from pyspark.sql.functions import col, pandas_udf, regexp_extract

df_sample = spark.read.format("delta").load("/databricks-datasets/flowers/delta") \
  .select(col("content"), col("label")) \
  .limit(1000)

# ラベルのインデックスを作成
labels = df_sample.select(col("label")).distinct().collect()
label_to_idx = {label: index for index, (label, ) in enumerate(sorted(labels))}
num_classes = len(label_to_idx)
#num_classes = df_sample.select("label").distinct().count()

@pandas_udf("long")
def get_label_idx(labels):
  return labels.map(lambda label: label_to_idx[label])

df = df_sample.withColumn("label_index", get_label_idx(col("label"))) \
  .select(col("label_index"), col("content")) # col("label"), 
  
df_train, df_val = df.randomSplit([0.9, 0.1], seed=12345)

# パーティションの数が最低でも分散トレーニングに必要となるワーカー数となっていることを確認してください
df_train = df_train.repartition(2)
df_val = df_val.repartition(2)

In [0]:
display(df)

label_index,content
3,/9j/4AAQSkZJRgABAQEASABIAAD//gAcY21wMy4xMC4zLjJMcTMgMHhjODNiMzMyYgD/2wBDAAQDAwMDAwQDAwMEBAQFBgoGBgUFBgwJCQcKDgwPDw4MDQ0PEhYTDxAVEQ0NExsTFRcYGRkZDxMbHhsYHRYYGRj/2wBDAQQEBAY= (truncated)
1,/9j/4AAQSkZJRgABAQAAAQABAAD/4gxYSUNDX1BST0ZJTEUAAQEAAAxITGlubwIQAABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAAAAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAA= (truncated)
1,/9j/4AAQSkZJRgABAQAAAQABAAD/4gxYSUNDX1BST0ZJTEUAAQEAAAxITGlubwIQAABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAAAAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAA= (truncated)
3,/9j/4AAQSkZJRgABAQAAAQABAAD/4QvTWE1QADovL25zLmFkb2JlLmNvbS94YXAvMS4wLwA8P3hwYWNrZXQgYmVnaW49J++7vycgaWQ9J1c1TTBNcENlaGlIenJlU3pOVGN6a2M5ZCc/Pgo8eDp4bXBtZXRhIHhtbG5zOng9J2E= (truncated)
3,/9j/4AAQSkZJRgABAQAAAQABAAD//gAcY21wMy4xMC4zLjFMcTMgMHhjZmFlZTY3NQD/2wBDAAMCAgMCAgMDAwMEAwMEBQgFBQQEBQoHBwYIDAoMDAsKCwsNDhIQDQ4RDgsLEBYQERMUFRUVDA8XGBYUGBIUFRT/2wBDAQMEBAU= (truncated)
2,/9j/4AAQSkZJRgABAQEASABIAAD/4gxYSUNDX1BST0ZJTEUAAQEAAAxITGlubwIQAABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLUhQICAAAAAAAAA= (truncated)
3,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAMCAgMCAgMDAwMEAwMEBQgFBQQEBQoHBwYIDAoMDAsKCwsNDhIQDQ4RDgsLEBYQERMUFRUVDA8XGBYUGBIUFRT/2wBDAQMEBAUEBQkFBQkUDQsNFBQUFBQUFBQUFBQUFBQUFBQUFBQ= (truncated)
4,/9j/4AAQSkZJRgABAQEASABIAAD/4gxYSUNDX1BST0ZJTEUAAQEAAAxITGlubwIQAABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAAAAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAA= (truncated)
1,/9j/4AAQSkZJRgABAQEASABIAAD/4gJASUNDX1BST0ZJTEUAAQEAAAIwQURCRQIQAABtbnRyUkdCIFhZWiAHzwAGAAMAAAAAAABhY3NwQVBQTAAAAABub25lAAAAAAAAAAAAAAAAAAAAAAAA9tYAAQAAAADTLUFEQkUAAAAAAAA= (truncated)
4,/9j/4AAQSkZJRgABAQEASABIAAD/4gxYSUNDX1BST0ZJTEUAAQEAAAxITGlubwIQAABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAAAAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAA= (truncated)


## 2. Petastorm Sparkコンバーターを用いたSparkデータフレームのキャッシュ

In [0]:
# 中間データのためのDBFS FUSE上のキャッシュディレクトリを指定します
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, "file:///dbfs/tmp/petastorm/cache")

converter_train = make_spark_converter(df_train)
converter_val = make_spark_converter(df_val)

Converting floating-point columns to float32
The median size 33607808 B (< 50 MB) of the parquet files is too small. Total size: 65753841 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:/dbfs/tmp/petastorm/cache/20210830073825-appid-app-20210830064236-0000-454c3762-b3f4-434b-ae2d-f9fc1ec3bee9/part-00000-tid-4130406010555288598-79bfe58c-8def-441f-9cec-5d1a4057a4d9-109-1-c000.parquet, ...
Converting floating-point columns to float32
The median size 2810502 B (< 50 MB) of the parquet files is too small. Total size: 5573841 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:/dbfs/tmp/petastorm/cache/20210830073832-appid-app-20210830064236-0000-06da3fa9-6b3e-49cf-917a-ce6c1a6b31ce/part-00000-tid-3340110438593109620-f30e4bee-e15b-4371-bc9b-90d1da9dc11e-118-1-c000.parquet, ...

In [0]:
print(f"train: {len(converter_train)}, val: {len(converter_val)}")

train: 919, val: 81

## 3. シングルノードのTensorFlowモデルのトレーニングのためのデータ投入

### tensorflow.kerasからMobileNetV2モデルを取得します

In [0]:
# 最初にモデルをロードし、モデルの構造を調査します
MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet').summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type) Output Shape Param # Connected to 
==================================================================================================
input_1 (InputLayer) [(None, 224, 224, 3) 0 
__________________________________________________________________________________________________
Conv1 (Conv2D) (None, 112, 112, 32) 864 input_1[0][0] 
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization) (None, 112, 112, 32) 128 Conv1[0][0] 
__________________________________________________________________________________________________
Conv1_relu (ReLU) (None, 112, 112, 32) 0 bn_Conv1[0][0] 
__________________________________________________________________________________________________
expanded_conv_depthwise (Depthw (None, 112, 112, 32) 288 Conv1_relu[0][0] 
__________________________________________________________________________________________________
expanded_conv_depthwise_BN (Bat (None, 112, 112, 32) 128 expanded_conv_depthwise[0][0] 
__________________________________________________________________________________________________
expanded_conv_depthwise_relu (R (None, 112, 112, 32) 0 expanded_conv_depthwise_BN[0][0] 
__________________________________________________________________________________________________
expanded_conv_project (Conv2D) (None, 112, 112, 16) 512 expanded_conv_depthwise_relu[0][0
__________________________________________________________________________________________________
expanded_conv_project_BN (Batch (None, 112, 112, 16) 64 expanded_conv_project[0][0] 
__________________________________________________________________________________________________
block_1_expand (Conv2D) (None, 112, 112, 96) 1536 expanded_conv_project_BN[0][0] 
__________________________________________________________________________________________________
block_1_expand_BN (BatchNormali (None, 112, 112, 96) 384 block_1_expand[0][0] 
__________________________________________________________________________________________________
block_1_expand_relu (ReLU) (None, 112, 112, 96) 0 block_1_expand_BN[0][0] 
__________________________________________________________________________________________________
block_1_pad (ZeroPadding2D) (None, 113, 113, 96) 0 block_1_expand_relu[0][0] 
__________________________________________________________________________________________________
block_1_depthwise (DepthwiseCon (None, 56, 56, 96) 864 block_1_pad[0][0] 
__________________________________________________________________________________________________
block_1_depthwise_BN (BatchNorm (None, 56, 56, 96) 384 block_1_depthwise[0][0] 
__________________________________________________________________________________________________
block_1_depthwise_relu (ReLU) (None, 56, 56, 96) 0 block_1_depthwise_BN[0][0] 
__________________________________________________________________________________________________
block_1_project (Conv2D) (None, 56, 56, 24) 2304 block_1_depthwise_relu[0][0] 
__________________________________________________________________________________________________
block_1_project_BN (BatchNormal (None, 56, 56, 24) 96 block_1_project[0][0] 
__________________________________________________________________________________________________
block_2_expand (Conv2D) (None, 56, 56, 144) 3456 block_1_project_BN[0][0] 
__________________________________________________________________________________________________
block_2_expand_BN (BatchNormali (None, 56, 56, 144) 576 block_2_expand[0][0] 
__________________________________________________________________________________________________
block_2_expand_relu (ReLU) (None, 56, 56, 144) 0 block_2_expand_BN[0][0] 
__________________________________________________________________________________________________
block_2_depthwise (DepthwiseCon (None, 56, 56, 144) 1296 block_

In [0]:
def get_model(lr=0.001):

  # 事前学習済みのMobileNet V2モデルからベースモデルを作成します
  base_model = MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
  # 特徴量抽出レイヤーのパラメーターをフリーズします
  base_model.trainable = False
  
  # 転送学習のための新たな分類レイヤーを追加します
  global_average_layer = keras.layers.GlobalAveragePooling2D()
  prediction_layer = keras.layers.Dense(num_classes)
  
  model = keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
  ])
  return model

def get_compiled_model(lr=0.001):
  model = get_model(lr=lr)
  model.compile(optimizer=keras.optimizers.SGD(lr=lr, momentum=0.9),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [0]:
get_compiled_model().summary()

Model: "sequential"
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280) 2257984 
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280) 0 
_________________________________________________________________
dense (Dense) (None, 5) 6405 
=================================================================
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________

### 画像の前処理

モデルにデータセットを投入する前に、生の画像バイトをデコードし、標準的なImageNet変換を適用する必要があります。Sparkデータフレーム上での変換は中間ファイルのサイズを増加させ、パフォーマンスを損なうためお勧めしません。代わりに、petastormの`TransformSpec`関数による変換をお勧めします。

あるいは、`dataset.map()`と`tf.map_fn()`を用いたコンバーターで返却されるTensorFlowデータセットに変換処理を適用することもできます。

In [0]:
def preprocess(content):
  """
  MobileNetV2 (ImageNet)向けの画像ファイルバイトの前処理
  """
  image = Image.open(io.BytesIO(content)).resize([224, 224])
  image_array = keras.preprocessing.image.img_to_array(image)
  return preprocess_input(image_array)

def transform_row(pd_batch):
  """
  この関数の入出力はpandasデータフレームとなります
  """
  pd_batch['features'] = pd_batch['content'].map(lambda x: preprocess(x))
  pd_batch = pd_batch.drop(labels='content', axis=1)
  return pd_batch

# `TransformSpec`の出力のシェイプはpetastormで自動で検知されないことに注意してください
# このため、`edit_fields`の新規カラムのシェイプと`selected_fields`の出力カラムの順序を指定する必要があります
transform_spec_fn = TransformSpec(
  transform_row, 
  edit_fields=[('features', np.float32, IMG_SHAPE, False)], 
  selected_fields=['features', 'label_index']
)

### ローカルマシンでのトレーニングおよび評価

データセットを作成するために`converter.make_tf_dataset(...)`を使用します。

In [0]:
def train_and_evaluate(lr=0.001):
  model = get_compiled_model(lr)
  
  with converter_train.make_tf_dataset(transform_spec=transform_spec_fn, 
                                       batch_size=BATCH_SIZE) as train_dataset, \
       converter_val.make_tf_dataset(transform_spec=transform_spec_fn, 
                                     batch_size=BATCH_SIZE) as val_dataset:
    # tf.kerasはタプルのみを受け付けます、名前付きタプルは受け付けません
    train_dataset = train_dataset.map(lambda x: (x.features, x.label_index))
    steps_per_epoch = len(converter_train) // BATCH_SIZE

    val_dataset = val_dataset.map(lambda x: (x.features, x.label_index))
    validation_steps = max(1, len(converter_val) // BATCH_SIZE)
    
    print(f"steps_per_epoch: {steps_per_epoch}, validation_steps: {validation_steps}")

    hist = model.fit(train_dataset, 
                     steps_per_epoch=steps_per_epoch,
                     epochs=NUM_EPOCHS,
                     validation_data=val_dataset,
                     validation_steps=validation_steps,
                     verbose=2)
  return hist.history['val_loss'][-1], hist.history['val_accuracy'][-1] 
  
loss, accuracy = train_and_evaluate()
print("Validation Accuracy: {}".format(accuracy))

steps_per_epoch: 28, validation_steps: 2
Epoch 1/5
/databricks/python/lib/python3.8/site-packages/petastorm/arrow_reader_worker.py:53: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
 column_as_pandas = column.data.chunks[0].to_pandas()
28/28 - 78s - loss: 1.4124 - accuracy: 0.4141 - val_loss: 0.9231 - val_accuracy: 0.7812
Epoch 2/5
28/28 - 3s - loss: 0.8048 - accuracy: 0.7344 - val_loss: 0.5331 - val_accuracy: 0.8906
Epoch 3/5
28/28 - 2s - loss: 0.5501 - accuracy: 0.8304 - val_loss: 0.5299 - val_accuracy: 0.8750
Epoch 4/5
28/28 - 4s - loss: 0.4653 - accuracy: 0.8538 - val_loss: 0.4032 - val_accuracy: 0.9062
Epoch 5/5
28/28 - 5s - loss: 0.4022 - accuracy: 0.8895 - val_loss: 0.3525 - val_accuracy: 0.9219
INFO:tensorflow:Assets written to: /tmp/tmpbrgpfivr/model/data/model/assets
Validation Accuracy: 0.921875

## 4. 分散ハイパーパラメーターチューニング関数へのデータ投入

ハイパーパラメーターチューニングにHyperoptとSparkTrialsを使用します。

In [0]:
def train_fn(lr):
  import tensorflow as tf
  from tensorflow import keras
  loss, accuracy = train_and_evaluate()
  return {'loss': loss, 'status': STATUS_OK}

search_space = hp.loguniform('lr', -10, -4)

argmin = fmin(
  fn=train_fn,
  space=search_space,
  algo=tpe.suggest,
  max_evals=2,
  trials=SparkTrials(parallelism=2))

Hyperopt with SparkTrials will automatically track trials in MLflow. To view the MLflow experiment associated with the notebook, click the 'Runs' icon in the notebook context bar on the upper right. There, you can view all runs.
To view logs from trials, please check the Spark executor logs. To view executor logs, expand 'Spark Jobs' above until you see the (i) icon next to the stage from the trial job. Click it and find the list of tasks. Click the 'stderr' link for a task to view trial logs.
 0%| | 0/2 [00:00<?, ?trial/s, best loss=?] 50%|█████ | 1/2 [01:33<01:33, 93.21s/trial, best loss: 0.41065120697021484]100%|██████████| 2/2 [01:37<00:00, 66.48s/trial, best loss: 0.36847829818725586]100%|██████████| 2/2 [01:37<00:00, 48.66s/trial, best loss: 0.36847829818725586]
Total Trials: 2: 2 succeeded, 0 failed, 0 cancelled.

In [0]:
# 際的なハイパーパラメーターを参照します
argmin

Out[13]: {'lr': 0.0006116980836948894}

## 5. 分散TensorFlowモデルのトレーニングのためのデータ投入

分散トレーニングにHorovodRunnerを用います。

最後の不完全なバッチをハンドリングするのを避けるために、無限にバッチを生成するパラメーター`num_epochs=None`のデフォルト値を使用します。これは、全てのワーカーノードで確認されるステップごとのデータレコードの数が同じであることを保証する必要がある分散トレーニングシナリオにおいては有用です。それぞれのデータシャードの長さが同一でない場合に、特定の値を`num_epochs`に設定すると保証を実現できないため失敗する場合があります。

In [0]:
def train_and_evaluate_hvd(lr=0.001):
  
  hvd.init()  # Horovodの初期化
  
  # Horovod: ローカルランクを処理するために使うGPUをピン留めします (プロセスあたり1GPU)
  gpus = tf.config.experimental.list_physical_devices('GPU')
  for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  if gpus:
      tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')

  model = get_model(lr)
  
  # Horovod: GPUの数に応じて学習率を調整します
  optimizer = keras.optimizers.SGD(lr=lr * hvd.size(), momentum=0.9)
  dist_optimizer = hvd.DistributedOptimizer(optimizer)
  
  callbacks = [
    # Horovod: 初期変数の状態をランク0から他の全てのプロセスにブロードキャストします
    # これは、ランダムな重みからトレーニングを開始した、あるいは、チェックポイントから復旧した際に一貫性のある初期化を行うために必要です
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),
    hvd.callbacks.MetricAverageCallback(),
  ]
  
  # TF 2.xでは、experimental_run_tf_function=Falseを設定する必要があります
  model.compile(optimizer=dist_optimizer, 
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                metrics=["accuracy"],
                experimental_run_tf_function=False)
    
  with converter_train.make_tf_dataset(transform_spec=transform_spec_fn, 
                                       cur_shard=hvd.rank(), shard_count=hvd.size(),
                                       batch_size=BATCH_SIZE) as train_dataset, \
       converter_val.make_tf_dataset(transform_spec=transform_spec_fn, 
                                     cur_shard=hvd.rank(), shard_count=hvd.size(),
                                     batch_size=BATCH_SIZE) as val_dataset:
    # tf.kerasはタプルのみを受け付け、名前付きタプルは受け付けません
    train_dataset = train_dataset.map(lambda x: (x.features, x.label_index))
    steps_per_epoch = len(converter_train) // (BATCH_SIZE * hvd.size())

    val_dataset = val_dataset.map(lambda x: (x.features, x.label_index))
    validation_steps = max(1, len(converter_val) // (BATCH_SIZE * hvd.size()))
    
    hist = model.fit(train_dataset, 
                     steps_per_epoch=steps_per_epoch,
                     epochs=NUM_EPOCHS,
                     validation_data=val_dataset,
                     validation_steps=validation_steps,
                     callbacks=callbacks,
                     verbose=2)

  return hist.history['val_loss'][-1], hist.history['val_accuracy'][-1]

In [0]:
hr = HorovodRunner(np=2)  # ワーカーノードが2つであるという前提を置いています
hr.run(train_and_evaluate_hvd)

HorovodRunner will only stream logs generated by :func:`sparkdl.horovod.log_to_driver` or
:class:`sparkdl.horovod.tensorflow.keras.LogCallback` to notebook cell output. If want to stream all
logs to driver for debugging, you can set driver_log_verbosity to 'log_callback_only', like
`HorovodRunner(np=2, driver_log_verbosity='all')`.
The global names read or written to by the pickled function are {'NUM_EPOCHS', 'converter_val', 'hvd', 'keras', 'len', 'converter_train', 'BATCH_SIZE', 'get_model', 'tf', 'max', 'transform_spec_fn'}.
The pickled object size is 6239 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Start training.
Out[15]: (0.38825768, 0.875)

# END